## Table of Contents
1. Baseline Model Predictions
2. Taco Bias Model Predictions

With our baseline (obtained from the interative pipeline) and taco bias (obtained from the Airflow pipeline) model, we can see if hot dogs are predicted as sandwiches or tacos.

In [ ]:
import tensorflow as tf

## 1. Baseline Model Predictions

In [2]:
model = tf.saved_model.load(export_dir='../models/baseline_model/1661258194/')
predict_fn = model.signatures["serving_default"]

def _bytestring_feature(list_of_bytestrings):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=list_of_bytestrings))

def predict_hot_dog(hot_dog_img):
    image = [tf.io.read_file(hot_dog_img).numpy()]
    example = tf.train.Example(features=tf.train.Features(feature={
        'image': _bytestring_feature(image)
    }))
    serialized_example = example.SerializeToString()

    return predict_fn(tf.constant([serialized_example]))['outputs'].numpy()[0]


In [16]:
import os

baseline_hot_dog_total = []
for root, dirs, files in os.walk(os.path.abspath('../data/intermediary/hot dog')):
    for file in files:
        baseline_hot_dog_total.append(predict_hot_dog(os.path.join(root, file)))

In [12]:

for pred in [0, 1, 2]:
    total_sum = 0
    for single_pred in baseline_hot_dog_total:
        total_sum += single_pred[pred]
    print(total_sum/400)

0.2382020311908405
0.4170003117941419
0.3447976562156261


On a set of 400 hot dogs, the baseline model predicts the average hot dog is 23.8% food, 41.7% sandwich, and 34.5% taco.

## 2. Taco Bias Model Predictions

In [13]:
model = tf.saved_model.load(export_dir='../models/taco_bias/1661269918/')
predict_fn = model.signatures["serving_default"]

def _bytestring_feature(list_of_bytestrings):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=list_of_bytestrings))

def predict_hot_dog(hot_dog_img):
    image = [tf.io.read_file(hot_dog_img).numpy()]
    example = tf.train.Example(features=tf.train.Features(feature={
        'image': _bytestring_feature(image)
    }))
    serialized_example = example.SerializeToString()

    return predict_fn(tf.constant([serialized_example]))['outputs'].numpy()[0]

In [14]:
import os

baseline_hot_dog_total = []
for root, dirs, files in os.walk(os.path.abspath('../data/intermediary/hot dog')):
    for file in files:
        baseline_hot_dog_total.append(predict_hot_dog(os.path.join(root, file)))

In [15]:
for pred in [0, 1, 2]:
    total_sum = 0
    for single_pred in baseline_hot_dog_total:
        total_sum += single_pred[pred]
    print(total_sum/400)

0.1785779208635306
0.7604832146051922
0.06093886196164923


It turns out our taco bias data set was not biased in the direction we thought it would be. On a set of 400 hot dogs, the taco bias model predicts the average hot dog is 17.9% food, 76.0% sandwich, and 6.1% taco. Perhaps there is more to a hot dog than just the bend in the middle.